In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
## Load the dataset
data = pd.read_csv("../data/Churn_Modelling.csv")
data.head()

In [ ]:
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data

In [ ]:
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])
data

## Onehot encode 'Geography'


In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]]).toarray()
geo_encoder

In [ ]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

In [ ]:
geo_encoded_df = pd.DataFrame(
    geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(["Geography"])
)
geo_encoded_df

In [ ]:
data = pd.concat([data.drop("Geography", axis=1), geo_encoded_df], axis=1)
data.head()

In [ ]:
with open("../models/label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("../models/onehot_encoder_geo.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)

In [ ]:
data.head()

### train-test split


In [ ]:
X = data.drop("Exited", axis=1)
y = data["Exited"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
with open("../models/scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

# ANN Implementation


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
input_shape = X_train.shape

In [ ]:
model = Sequential(
    [
        Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
        Dense(32, activation="relu"),
        Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback],
)

In [ ]:
model.save("../models/model.keras")

In [103]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [104]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 15784), started 0:00:31 ago. (Use '!kill 15784' to kill it.)